# Importing required libs
* We use tensorflow keras in our research
* CV for reading pictures

In [ ]:
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer, LabelEncoder
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import models
from tensorflow.keras import Sequential

# Loading images
* loading images to compliant to the facenet keras face feature extrating model
there are 165 images about 15 people with means average 11 images for each of the people, which isn't a big dataset qualifies to our research scope of fewer-shoot based face indetification. 

In [ ]:
y=[]
X=[]


for dirname, _, filenames in os.walk('/kaggle/input/yale-face-database/data'):
    
    fc=0
    for filename in filenames:
       
        if filename!="Readme.txt":
            b=plt.imread(os.path.join(dirname, filename))
       
            try:
                b= cv2.cvtColor(b,cv2.COLOR_GRAY2RGB) 
            except:
                pass
            b=cv2.resize(b, (160,160),interpolation = cv2.INTER_AREA)

            if b.shape==(160,160,3) :
                fc=fc+1
                #print(int(filename[7:9]))
                y.append(int(filename[7:9]))
                X.append(b)
            else:
                print(face.shape)
    print(fc,len(X),len(y))


In [ ]:
x_cluster = np.array(X)

print(type(x_cluster))

print(x_cluster.shape)

The minimum and maximum values are 0 and 255 respectively. In the RGB color space the red, green and blue use 8 bits each which have integer values from 0 to 255. So the total number of possible colors is 256*256*256 = 16777216. Sounds astonishing?

In [ ]:
# Data Normalization
# Conversion to float
x_cluster = x_cluster.astype('float32') 
x_cluster = x_cluster/255.0

In [ ]:
# Checking the minimum and maximum values of x_cluster
print(x_cluster.min())
print(x_cluster.max())

The input data have to be converted from 3 dimensional format to 2 dimensional format to be fed into the K-Means Clustering algorithm. Hence the input data has to be reshaped.

In [ ]:
X_cluster = x_cluster.reshape(len(x_cluster),-1)

In [ ]:
print(X_cluster.shape)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
#select different clusters, after analysis 3 is the best for this dataset
total_clusters = 3
# Initialize the K-Means model
kmeans = MiniBatchKMeans(n_clusters = total_clusters)
# Fitting the model to training set
kmeans.fit(X_cluster)

In [ ]:
c_index=kmeans.labels_

In [ ]:
(unique, counts) = np.unique(c_index, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
counts

# Dividing the images based on the clusters
* The images are divided into 3 cluster with 80, 29 and 56 images respectly in this instance run.
* However, each time the images number will be different based on the runtime clustering instance, when you run the code you can get completly different numbers for each cluster

In [ ]:
c_datag=[]
y_datag=[]
for i in range(0,total_clusters):
    c_data=[]
    y_data=[]
    for j in range(0, len(c_index)):
        #print(counts[i])
        if c_index[j] == i: 
            #print (i,j)
            c_data.append(X[j])
            y_data.append(y[j])
    print(len(c_data),len(y_data))
    c_datag.append(c_data)
    y_datag.append(y_data)

In [ ]:
print(len(c_datag[0]), len(c_datag[1]), len(c_datag[2]))

In [ ]:
image_data=[]
#for each cluster
for k in range(total_clusters):
    k_data=[]
    #check if any person belongs to this cluster, if yes, then add
    for i in range(1,16):
        person_=[]
        for u in range(len(c_datag[k])):
            if y_datag[k][u]==i:
                person_.append(c_datag[k][u])
        k_data.append([person_,i])
    image_data.append(k_data)

In [ ]:
len(image_data)

In [ ]:
print(len(image_data[0]), len(image_data[1]),len(image_data[2]))

In [ ]:
image_data[0][2][1]

In [ ]:
len(image_data[0][0][0])

# Generating sensitive attention image pairs
* The reason it is sensitive because that the image pair will be generated by considering the clusters. The clusters should contain images that have some common features in the images.
* The positive pairs are generated inside the same cluster but more importantly from different clusters, which means the same person's image pairs have different features e.g. wearing glasses or having different emotions or strong shadows and so on.
* The negtive pairs are generated focusing on inside the same cluster, which means the negtive pairs may contain some the common features. This is important as we try to make the model catching more sensitive features that can indentify different person.  

In [ ]:
NPair_X1 = []
NPair_X2 = []
PPair_X1 = []
PPair_X2 = []
Pair_y = []

In [ ]:
def generate_NPairs_samecluster(img_data,cluster_n):
    for k in range(cluster_n):
        #print(k)
        for kk in range(len(img_data[k])):
            #print(kk)
            if len(img_data[k][kk][0])>=1 and kk+1<len(img_data[k]):
                #print(kk+1)
                if img_data[k][kk][1]!= img_data[k][kk+1][1] and len(img_data[k][kk+1][0])>0:
                    NPair_X1.append(img_data[k][kk][0][0]) 
                    NPair_X2.append(img_data[k][kk+1][0][0])
                    Pair_y.append(1)                      

In [ ]:
def generate_PPairs_diffcluster(img_data,cluster_n):
    for k in range(cluster_n):
        for kk in range(len(img_data[k])):
            if len(img_data[k][kk][0])>=1:
                if k+1 < cluster_n:
                    for kkk in range(len(image_data[k+1])):
                        if img_data[k][kk][1]== img_data[k+1][kkk][1] and len(img_data[k+1][kkk][0])>0:
                            PPair_X1.append(img_data[k][kk][0][0]) 
                            PPair_X2.append(img_data[k+1][kkk][0][0])
                            Pair_y.append(0)

In [ ]:
generate_NPairs_samecluster(image_data,3)
len(NPair_X1),len(NPair_X2),len(Pair_y)

In [ ]:
generate_PPairs_diffcluster(image_data,3)

In [ ]:
len(PPair_X1),len(PPair_X2),len(Pair_y)

In [ ]:
PPair_X1[0]

# Visualise the negative pairs that are genterated from the same cluster

In [ ]:
fig=plt.figure(figsize=(20, 20))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    ax=fig.add_subplot(rows, columns, i)
    plt.imshow(np.concatenate([NPair_X1[i-1],NPair_X2[i-1]], axis=1))
    ax.title.set_text(str(Pair_y[i-1])+': negative_pair_same_cluster') 
    ax.set_xticks([]) 
    ax.set_yticks([])
plt.show()

In [ ]:
fig=plt.figure(figsize=(20, 20))
columns = 3
rows = 6
for i in range(1, columns*rows +1):
    ax=fig.add_subplot(rows, columns, i)
    plt.imshow(np.concatenate([PPair_X1[i-1],PPair_X2[i-1]], axis=1))
    ax.title.set_text(str(Pair_y[i+25-1])+': positive_pair_diff_cluster')
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

# General pairs normally

In [ ]:
#sort according to the person
data=[]
for i in range(1,16):
    person=[]
    for u in range(len(y)):
        if y[u]==i:
            person.append(X[u])
    data.append([person, i*len(person)])
    print(str(len(person))+" images  for class "+ str(i))

In [ ]:
import random 
random.seed(42)
newX1=[]
newX2=[]
newY=[]
c1 = 0
c2 = 0
for i in range(len(data)):
    parts= int(len(data[i][0])/2)
    #positive sampling from same person
    for u in range(len(data[i][0])):
        for k in range(int(parts/2)):
            g = random.randint(0,len(data[i])-1)
            if u!=k and u!=g:
                newX1.append(data[i][0][u])
                newX2.append(data[i][0][g])
                newY.append(0)
                c1=c1+1
    #print(k,c1)
    #negative sampling g from different person
    for uu in range(1,15):
        for uuu in range(int(len(data[uu]))):
            g = random.randint(0,len(data[i])-1)
            if uu!=i:
                newX1.append(data[i][0][g])
                newX2.append(data[uu][0][uuu])
                newY.append(1)
                c2=c2+1
print("Target values count: "+ str(np.unique(newY, return_counts=True)), c1+c2,c1,c2)

In [ ]:
fig=plt.figure(figsize=(20, 20))
columns = 10
rows = 10
for i in range(1, columns*rows +1):
    ax=fig.add_subplot(rows, columns, i)
    plt.imshow(np.concatenate([newX1[i-1],newX2[i-1]], axis=1))
    ax.title.set_text(str(newY[i-1])) 
    ax.set_xticks([]) 
    ax.set_yticks([])
plt.show()

# Pairs are generated
* In total there are 620 pairs are generated
* 313 positive pairs
* 307 negative pairs

In [ ]:
len(newX1)

In [ ]:
len(newX2)

# Pairs integration

In [ ]:
print(len(NPair_X1),len(PPair_X1))

In [ ]:
newX1= newX1 + NPair_X1
newX2= newX2 + NPair_X2
newX1= newX1 + PPair_X1
newX2= newX2 + PPair_X2
newY= newY+ Pair_y
print(len(newX1),len(newX2),len(newY))

In [ ]:
c = list(zip(newX1, newX2, newY))

random.shuffle(c)

a, b, y = zip(*c)

# Further feature extraction with PCA approach

In [ ]:
#np.array(a)
Xpca=np.array(a)
Xpca.shape

# Showing the example of PCA results

In [ ]:
Xpca=Xpca.reshape(Xpca.shape[0],Xpca.shape[1]*Xpca.shape[2]*Xpca.shape[3])
from sklearn.decomposition import PCA

faces_pca = PCA(n_components=0.9)
faces_pca.fit(Xpca)
fig, axes = plt.subplots(3,5,figsize=(9,3),
 subplot_kw={'xticks':[], 'yticks':[]},
 gridspec_kw=dict(hspace=0.01, wspace=0.01))
for i, ax in enumerate(axes.flat):
    ax.imshow(np.interp(faces_pca.components_[i], (min(faces_pca.components_[i]),max(faces_pca.components_[i])), (0, 255)).reshape(160,160,3).astype("int"))

In [ ]:
Xpca.shape

# Using facenet pre-trained model to encode the image pairs
* output 1 will be the same person image model, the output will be 0
* output 2 will be the different person image model, the output will be 1

In [ ]:
base_model = models.load_model('../input/keras-facenet-modelweights/keras-facenet/model/facenet_keras.h5')

base_model.load_weights("../input/keras-facenet-modelweights/keras-facenet/weights/facenet_keras_weights.h5")
base_model.trainable=False
output1=base_model.predict(np.array(a)/255)
output2=base_model.predict(np.array(b)/255)
indexes1=[i for i,x in enumerate(y) if x == 1]
indexes0=[i for i,x in enumerate(y) if x == 0]

# Looking for differences:
* get absolute values of features subtracted between two sample images
* sum all the absolute values for each array
* to see if there is a visual difference between positive and negative samples

1. Subtraction differences

In [ ]:
arr=[]
su=[]
for s in range(len(output1)):
    oo = np.abs(np.subtract(np.array(output1[s]),np.array(output2[s])))
    arr.append(oo)
    su.append(oo.sum())
    
a = np.array(su)
su1=list(a[indexes1])
su0=list(a[indexes0])

print(su1)

In [ ]:
print(len(su1))
print(len(su0))

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Subtraction distance")
axs[0].plot(list(range(len(su0))),su1[:len(su0)], list(range(len(su0))),su0)
axs[0].legend(["different people", "same person"])
axs[1].plot(list(range(len(newX1))),su)
axs[1].legend(["overall variation"])

In [ ]:
from scipy import stats
import statistics
print(stats.describe(su))
print("median "+str(statistics.median(su)))

2. Euclidean difference

In [ ]:
import scipy
euc=[]
for s in range(len(output1)):
    oo = scipy.spatial.distance.euclidean(np.array(output1[s]),np.array(output2[s]))
    euc.append(oo)
a = np.array(euc)
euc1=list(a[indexes1])
euc0=list(a[indexes0])

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Euclidean distance")
axs[0].plot(list(range(len(euc0))),euc1[:len(euc0)], list(range(len(euc0))),euc0)
axs[0].legend(["different people", "same person"])
#axs[0].title("Euclidean distance")
axs[1].plot(list(range(len(newX1))),euc)
axs[1].legend(["overall variation"])

In [ ]:
print(stats.describe(euc))
print("median "+str(statistics.median(euc)))

3. Cosine differences 

In [ ]:
cos=[]
for s in range(len(output1)):
    oo = scipy.spatial.distance.cosine(np.array(output1[s]),np.array(output2[s]))
    cos.append(oo)
a = np.array(cos)
cos1=list(a[indexes1])
cos0=list(a[indexes0])
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Cosine distance")
axs[0].plot(list(range(len(cos0))), cos1[:len(cos0)], list(range(len(cos0))),cos0)
axs[1].plot(list(range(len(newX1))),cos)

In [ ]:
print(stats.describe(cos))
print("median "+str(statistics.median(cos)))

In [ ]:
features=Model(base_model.input, base_model.output)
import tensorflow.keras.backend as K
import tensorflow

In [ ]:
v=Input((160,160,3))
p=Input((160,160,3))

featuresA=features(v)
featuresB=features(p)

In [ ]:
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
distance = L1_layer([featuresA, featuresB])

In [ ]:
#def distance(vecs):
    #x, y = vecs
    #x = K.l2_normalize(x, axis=-1)
    #y = K.l2_normalize(y, axis=-1)
    
    #return K.abs(x-y)

In [ ]:
#L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#L1_distance = L1_layer([encoded_l, encoded_r])

In [ ]:
#v=Input((160,160,3))
#p=Input((160,160,3))

#featuresA=features(v)
#featuresB=features(p)
#distance= Lambda(distance)([featuresA,featuresB])

# Building the training model 

In [ ]:
x= Dense(96, activation="relu")(distance)
x= Dropout(0.3)(x)
x= Dense(64)(x)
outputs = Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=[v,p],outputs=outputs)

# Dividing the data into train and test

In [ ]:
x_=[]
for i in range(len(a)):
    connect_a_b =[]
    connect_a_b.append(a[i])
    connect_a_b.append(b[i])
    x_.append(connect_a_b)
    
x_train,x_test,y_train,y_test=train_test_split(x_,y,test_size=0.20)


In [ ]:
x_train_a=[]
x_train_b=[]
x_test_a = []
x_test_b = []
for ob in x_train:
    x_train_a.append(ob[0])
    x_train_b.append(ob[1])
for ob in x_test:
    x_test_a.append(ob[0])
    x_test_b.append(ob[1])

In [ ]:
x_test_clu_a = x_test_a
x_test_clu_b = x_test_b
y_test_clu = y_test

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()
history=model.fit([np.array(x_train_a)/255, np.array(x_train_b)/255],np.array(y_train),validation_data=([np.array(x_test_a)/255, np.array(x_test_b)/255],np.array(y_test)),epochs=20, batch_size=32)

model1 = Model(inputs=[v,p],outputs=outputs)
model1.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model1.summary()
history1=model1.fit([np.array(x_train_a)/255, np.array(x_train_b)/255],np.array(y_train),validation_data=([np.array(x_test_s_a)/255, np.array(x_test_s_b)/255],np.array(y_test_s)),epochs=20, batch_size=16)

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Overfitting analysis")
axs[0].plot(list(range(1,21)), history.history['val_accuracy'], list(range(1,21)), history.history['accuracy'])

axs[0].title.set_text("Accuracy")
axs[0].legend(["validation accuracy", "training accuracy"])
axs[1].plot(list(range(1,21)), history.history['val_loss'], list(range(1,21)), history.history['loss'])
axs[1].title.set_text('Loss')
axs[1].legend(["validation loss", "trainig loss"])

# ######Without clustering process###

In [ ]:
y=[]
X=[]


for dirname, _, filenames in os.walk('/kaggle/input/yale-face-database/data'):
    
    fc=0
    for filename in filenames:
       
        if filename!="Readme.txt":
            b=plt.imread(os.path.join(dirname, filename))
       
            try:
                b= cv2.cvtColor(b,cv2.COLOR_GRAY2RGB) 
            except:
                pass
            b=cv2.resize(b, (160,160),interpolation = cv2.INTER_AREA)

            if b.shape==(160,160,3) :
                fc=fc+1
                #print(int(filename[7:9]))
                y.append(int(filename[7:9]))
                X.append(b)
            else:
                print(face.shape)
    print(fc,len(y))

In [ ]:
len(y)

In [ ]:
#sort according to the person
data=[]
for i in range(1,16):
    person=[]
    for u in range(len(y)):
        if y[u]==i:
            person.append(X[u])
    data.append([person, i*len(person)])
    print(str(len(person))+" images  for class "+ str(i))

In [ ]:
len(data)

In [ ]:
k = data[0][0][0]
np.array(k)
k.shape

import random 
random.seed(42)
newX1=[]
newX2=[]
newY=[]
c1 = 0
c2 = 0
for i in range(len(data)):
    parts= int(len(data[i][0])/2)
    #positive sampling from same person
    for u in range(parts):
        c1=c1+1
        newX1.append(data[i][0][u])
        newX2.append(data[i][0][u+parts])
        newY.append(0)
    
    #negative sampling g from different person
    for uu in range(parts):
        c2=c2+1
        numbers = list(range(0,i)) + list(range(i+1,11))
        r = random.choice(numbers)
        g = random.randint(0,len(data[r][0])-1)
        newX1.append(data[i][0][uu])
        newX2.append(data[r][0][g])
        newY.append(1)
print("Target values count: "+ str(np.unique(newY, return_counts=True)), c1+c2,c1,c2)

In [ ]:
import random 
random.seed(42)
newX1=[]
newX2=[]
newY=[]
c1 = 0
c2 = 0
for i in range(len(data)):
    parts= int(len(data[i][0])/2)
    #positive sampling from same person
    for u in range(len(data[i][0])):
        for k in range(int(parts/2)):
            g = random.randint(0,len(data[i])-1)
            if u!=k:
                newX1.append(data[i][0][u])
                newX2.append(data[i][0][g])
                newY.append(0)
                c1=c1+1
    #print(k,c1)
    #negative sampling g from different person
    for uu in range(1,15):
        for uuu in range(int(len(data[uu]))):
            g = random.randint(0,len(data[i])-1)
            if uu!=i:
                newX1.append(data[i][0][g])
                newX2.append(data[uu][0][uuu])
                newY.append(1)
                c2=c2+1
print("Target values count: "+ str(np.unique(newY, return_counts=True)), c1+c2,c1,c2)

In [ ]:
len(newX1)

Creat pair

In [ ]:
fig=plt.figure(figsize=(15, 15))
columns = 10
rows = 12
for i in range(1, columns*rows +1):
    ax=fig.add_subplot(rows, columns, i)
    
    plt.imshow(np.concatenate([newX1[i-1],newX2[i-1]], axis=1))
    ax.title.set_text(newY[i-1])
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

In [ ]:
c = list(zip(newX1, newX2, newY))

random.shuffle(c)

a, b, y = zip(*c)

In [ ]:
#np.array(a)
Xpca=np.array(a)
Xpca.shape

PCA

In [ ]:
Xpca=Xpca.reshape(Xpca.shape[0],Xpca.shape[1]*Xpca.shape[2]*Xpca.shape[3])
from sklearn.decomposition import PCA

faces_pca = PCA(n_components=0.9)
faces_pca.fit(Xpca)
fig, axes = plt.subplots(2,5,figsize=(9,3),
 subplot_kw={'xticks':[], 'yticks':[]},
 gridspec_kw=dict(hspace=0.01, wspace=0.01))
for i, ax in enumerate(axes.flat):
    ax.imshow(np.interp(faces_pca.components_[i], (min(faces_pca.components_[i]),max(faces_pca.components_[i])), (0, 255)).reshape(160,160,3).astype("int"))

In [ ]:
Xpca.shape

In [ ]:
base_model = models.load_model('../input/keras-facenet-modelweights/keras-facenet/model/facenet_keras.h5')

base_model.load_weights("../input/keras-facenet-modelweights/keras-facenet/weights/facenet_keras_weights.h5")
base_model.trainable=False
output1=base_model.predict(np.array(a)/255)
output2=base_model.predict(np.array(b)/255)
indexes1=[i for i,x in enumerate(y) if x == 1]
indexes0=[i for i,x in enumerate(y) if x == 0]

# **Looking for differences:**
* get absolute values of features subtracted between two sample images
* sum all the absolute values for each array
* to see if there is a visual difference between positive and negative samples

In [ ]:
arr=[]
su=[]
for s in range(len(output1)):
    oo = np.abs(np.subtract(np.array(output1[s]),np.array(output2[s])))
    arr.append(oo)
    su.append(oo.sum())
    
a = np.array(su)
su1=list(a[indexes1])
su0=list(a[indexes0])

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Sum differences")
axs[0].plot(list(range(392)),su1, list(range(300)),su0)
axs[0].legend(["different people", "same person"])
#axs[0].title("Euclidean distance")
axs[1].plot(list(range(692)),su)
axs[1].legend(["overall variation"])

In [ ]:
from scipy import stats
import statistics
print(stats.describe(su))
print("median "+str(statistics.median(su)))

In [ ]:
import scipy
euc=[]
for s in range(len(output1)):
    oo = scipy.spatial.distance.euclidean(np.array(output1[s]),np.array(output2[s]))
    euc.append(oo)
a = np.array(euc)
euc1=list(a[indexes1])
euc0=list(a[indexes0])

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Euclidean distance")
axs[0].plot(list(range(392)),euc1, list(range(300)),euc0)
axs[0].legend(["different people", "same person"])
#axs[0].title("Euclidean distance")
axs[1].plot(list(range(692)),euc)
axs[1].legend(["overall difference"])

In [ ]:
print(stats.describe(euc))
print("median "+str(statistics.median(euc)))

In [ ]:
cos=[]
for s in range(len(output1)):
    oo = scipy.spatial.distance.cosine(np.array(output1[s]),np.array(output2[s]))
    cos.append(oo)
a = np.array(cos)
cos1=list(a[indexes1])
cos0=list(a[indexes0])
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Cosine distance")
axs[0].plot(list(range(392)), cos1, list(range(300)),cos0)
#axs[0].title("Euclidean distance")
axs[1].plot(list(range(692)),cos)

In [ ]:
print(stats.describe(cos))
print("median "+str(statistics.median(cos)))

In [ ]:
c = list(zip(newX1, newX2, newY))

random.shuffle(c)

a, b, y = zip(*c)

In [ ]:
features=Model(base_model.input, base_model.output)
import tensorflow.keras.backend as K
import tensorflow

In [ ]:
def distance(vecs):
    x, y = vecs
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    
    return K.abs(x-y)

In [ ]:
v=Input((160,160,3))
p=Input((160,160,3))

featuresA=features(v)
featuresB=features(p)
distance= Lambda(distance)([featuresA,featuresB])

In [ ]:
x= Dense(96, activation="relu")(distance)
x= Dropout(0.3)(x)
x= Dense(64)(x)
outputs = Dense(1, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=[v,p],outputs=outputs)

In [ ]:
len(np.array(a))

In [ ]:
x_=[]
for i in range(len(a)):
    connect_a_b =[]
    connect_a_b.append(a[i])
    connect_a_b.append(b[i])
    x_.append(connect_a_b)
    
x_train,x_test,y_train,y_test=train_test_split(x_,y,test_size=0.20)

In [ ]:
print('training size: ', len(x_train), 'testing size: ', len(x_test))

In [ ]:
x_train_a=[]
x_train_b=[]
x_test_a = []
x_test_b = []
for ob in x_train:
    x_train_a.append(ob[0])
    x_train_b.append(ob[1])
for ob in x_test:
    x_test_a.append(ob[0])
    x_test_b.append(ob[1])

Save the testing data for exchanging testing

In [ ]:
x_test_s_a=x_test_a
x_test_s_b=x_test_b
y_test_s = y_test

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()
#history=model.fit([np.array(a)[:120]/255, np.array(b)[:120]/255],np.array(y)[:120],validation_data=([np.array(a)[120:]/255, np.array(b)[120:]/255],np.array(y)[120:]), epochs=10, batch_size=16)
history=model.fit([np.array(x_train_a)/255, np.array(x_train_b)/255],np.array(y_train),validation_data=([np.array(x_test_a)/255, np.array(x_test_b)/255],np.array(y_test)),epochs=20, batch_size=32)

In [ ]:
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(18, 4)
fig.suptitle("Overfitting analysis")
axs[0].plot(list(range(1,21)), history.history['val_accuracy'], list(range(1,21)), history.history['accuracy'])

axs[0].title.set_text("Accuracy")
axs[0].legend(["validation accuracy", "training accuracy"])
axs[1].plot(list(range(1,21)), history.history['val_loss'], list(range(1,21)), history.history['loss'])
axs[1].title.set_text('Loss')
axs[1].legend(["validation loss", "trainig loss"])